# Сверточная нейронная сеть на PyTorch: пошаговое руководство

https://neurohive.io/ru/tutorial/cnn-na-pytorch/

In [40]:
import torch
import torch.nn as nn 

from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [41]:
num_epochs = 5 
num_classes = 10 
batch_size = 100 
learning_rate = 0.001
DATA_PATH = 'data\mnist'
MODEL_STORE_PATH = 'models'

## Загрузка датасета

In [42]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) 

train_dataset = datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True) 
test_dataset = datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [43]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True) 
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Создание модели

In [44]:
class ConvNet(nn.Module): 
    def __init__(self): 
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential( nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), 
        nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
        nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(7 * 7 * 64, 1000) 
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        return out    

## Обучение модели

In [45]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Тренировка

In [46]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.2440, Accuracy: 94.00%
Epoch [1/5], Step [200/600], Loss: 0.1752, Accuracy: 96.00%
Epoch [1/5], Step [300/600], Loss: 0.0829, Accuracy: 99.00%
Epoch [1/5], Step [400/600], Loss: 0.0568, Accuracy: 97.00%
Epoch [1/5], Step [500/600], Loss: 0.1813, Accuracy: 98.00%
Epoch [1/5], Step [600/600], Loss: 0.0565, Accuracy: 99.00%
Epoch [2/5], Step [100/600], Loss: 0.0495, Accuracy: 98.00%
Epoch [2/5], Step [200/600], Loss: 0.1115, Accuracy: 95.00%
Epoch [2/5], Step [300/600], Loss: 0.1435, Accuracy: 94.00%
Epoch [2/5], Step [400/600], Loss: 0.1035, Accuracy: 96.00%
Epoch [2/5], Step [500/600], Loss: 0.0207, Accuracy: 100.00%
Epoch [2/5], Step [600/600], Loss: 0.0332, Accuracy: 99.00%
Epoch [3/5], Step [100/600], Loss: 0.1464, Accuracy: 95.00%
Epoch [3/5], Step [200/600], Loss: 0.0557, Accuracy: 97.00%
Epoch [3/5], Step [300/600], Loss: 0.1591, Accuracy: 97.00%
Epoch [3/5], Step [400/600], Loss: 0.0638, Accuracy: 96.00%
Epoch [3/5], Step [500/600], Loss: 0.04

## Тестирование

In [47]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Сохраняем модель и строим график
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.17 %
